<a href="https://colab.research.google.com/github/tejatanush/Kaggle-Handwritten-digit-recognizer/blob/main/kaggle_OCR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Description:**
This model is to recognize handwritten number.This is a kaggle competition.                                                              
https://www.kaggle.com/competitions/digit-recognizer

# 1. Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split

# 2. Import Dataset

This dataset contains train and test sets. Train set is of 785 columns of which 784 columns are pixels and 1 column is label from 0 to 9. Test set is of 784 columns with only pixels. we shoul predict the label for each row based on pixels.

https://www.kaggle.com/competitions/digit-recognizer/data

In [2]:
dataset=pd.read_csv("train.csv")
test_data=pd.read_csv("test.csv")
dataset.head()


,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
test_data.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Unique values in dataset

In [4]:
unique_values=dataset['label'].unique()
print(unique_values)

[1 0 4 7 3 5 8 9 2 6]


We can see that there are 10 unique values

# Checking for missing values

In [5]:
missing_values=dataset.isnull().sum()
print(missing_values)

label       0
pixel0      0
pixel1      0
pixel2      0
pixel3      0
           ..
pixel779    0
pixel780    0
pixel781    0
pixel782    0
pixel783    0
Length: 785, dtype: int64


There are no any missing values in dataset

In [6]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42000 entries, 0 to 41999
Columns: 785 entries, label to pixel783
dtypes: int64(785)
memory usage: 251.5 MB


In [7]:
len(test_data)

28000

In [8]:
x=dataset.iloc[:,1:].values
y=dataset.iloc[:,0].values
test_data=test_data.iloc[:,:].values

In [9]:
x.astype('int')


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [10]:
y.astype('int')

array([1, 0, 1, ..., 7, 6, 9])

In [11]:
test_data.astype('int')

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

We had converted all the values in pixels are integers

# 3. Split in to training and validation sets

We split our dsata in to training and validation datasets in ratio of 4:1

In [12]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42)

Make them in to the shape of (28,28,1) because there are 784 pixels which will be the image shape of 28x28 and 1 is for gray scale image

In [13]:
x_train=x_train.reshape(-1,28,28,1)
x_val=x_val.reshape(-1,28,28,1)
test_data=test_data.reshape(-1,28,28,1)

Convert labels in to categorical

In [14]:
from keras.utils import to_categorical

y_train = to_categorical(y_train, num_classes=10)
y_val = to_categorical(y_val, num_classes=10)

In [15]:
print(x_train.shape)
print(y_train.shape)

(33600, 28, 28, 1)
(33600, 10)


Scale the values in between 0 and 1 for every pixel

In [17]:
x_train=x_train/255.0
x_val=x_val/255.0
test_data=test_data/255.0

# 4. Build a model

**Create Model:** To build a model we must extract features from the dataset by using maxpooling, convolution layers  with a good activation to remove linearity and then add classification layer with 10 nuerons because of 10 classes with softmax activation.

In [31]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D,AveragePooling2D
from tensorflow.keras.datasets import mnist
#from tensorflow.keras.applications import LeNet
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

In [32]:
model = Sequential([
    Conv2D(128, kernel_size=(5, 5), activation='tanh', input_shape=(28, 28, 1)),
    AveragePooling2D(pool_size=(2,2)),
    Conv2D(64, kernel_size=(5, 5), activation='tanh'),
    AveragePooling2D(pool_size=(2,2)),
    Flatten(),
    Dense(100, activation='tanh'),
    Dense(50, activation='tanh'),
    Dense(10, activation='softmax')
])



/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


**Compile Model:** Compile the model with Adam optimizer with learning rate of 0.0001 and categorical cross  entropy loss function to determine the difference between real and learned features of data. To evaluate metrics add accuracy so that we can understand the accuracy between predicted and real values.

In [33]:
# Compile model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

**Fit Model:** Fit the model with x_train,y_train and validation data with x_val and y_val.Keep the batch_size to 32

In [35]:
model.fit(x_train, y_train, batch_size=32, epochs=10, validation_data=(x_val, y_val))

Epoch 1/10
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 72s 68ms/step - accuracy: 0.7911 - loss: 0.8555 - val_accuracy: 0.9274 - val_loss: 0.2764
Epoch 2/10
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 72s 68ms/step - accuracy: 0.9367 - loss: 0.2351 - val_accuracy: 0.9512 - val_loss: 0.1736
Epoch 3/10
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 81s 67ms/step - accuracy: 0.9595 - loss: 0.1475 - val_accuracy: 0.9649 - val_loss: 0.1239
Epoch 4/10
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 82s 68ms/step - accuracy: 0.9719 - loss: 0.1022 - val_accuracy: 0.9699 - val_loss: 0.1051
Epoch 5/10
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 82s 68ms/step - accuracy: 0.9764 - loss: 0.0826 - val_accuracy: 0.9731 - val_loss: 0.0857
Epoch 6/10
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 82s 69ms/step - accuracy: 0.9819 - loss: 0.0665 - val_accuracy: 0.9762 - val_loss: 0.0772
Epoch 7/10
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 82s 68ms/step - accuracy: 0.9849 - loss: 0.0550 - val_accuracy: 0.9792 - val_loss: 0.0709
Epoch 8/10
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 84s 71ms/step - accuracy: 0.9874 -

# 5. predictions

Predict the results of test data with the model

In [36]:
y_test=model.predict(test_data)

875/875 ━━━━━━━━━━━━━━━━━━━━ 18s 20ms/step


In [37]:
print(y_test.shape)
ImageId=np.arange(1,28001)

(28000, 10)


In [38]:
print(y_test)

[[4.2302436e-05 6.3863321e-05 9.9966949e-01 ... 1.1082768e-04
  1.3133672e-05 7.2043349e-06]
 [9.9207890e-01 3.5499103e-05 9.8368409e-04 ... 6.6622428e-04
  1.2571382e-04 4.8170245e-04]
 [5.8895807e-06 3.3003056e-05 1.5415832e-04 ... 8.2336170e-05
  1.5828164e-03 9.9757791e-01]
 ...
 [8.8993039e-07 2.0285672e-05 2.0207530e-05 ... 4.3338132e-05
  3.4658704e-05 2.4290106e-04]
 [5.5260957e-06 4.1016847e-05 1.1632094e-05 ... 3.2444164e-04
  1.6111991e-04 9.9731755e-01]
 [1.6547507e-04 7.6765922e-05 9.9955451e-01 ... 9.0883004e-06
  7.7634380e-05 2.1769622e-05]]


These are the test results for the model which are predicted.

# 6. Create a submission file

In [39]:
submission=pd.DataFrame({'ImageId':ImageId,'Label':np.argmax(y_test,axis=1)})

In [40]:
submission.to_csv('submission.csv',index=False)

The csv file is downloaded which can be updated.

This submission file got a score of 0.97882 in kaggle digit recognizer competition.